In [1]:
import pandas as pd
import numpy as np

In [ ]:
import sqlite3

conn = sqlite3.connect('chinook.db')

query="""
select * from  Invoice 
"""
Inv=pd.read_sql_query(query, conn)


conn = sqlite3.connect('chinook.db')

query1="""
select * from  Customer 
"""
Cus=pd.read_sql_query(query1, conn)

In [27]:
all=pd.merge(Cus,Inv,on='CustomerId',how='inner')
all=all[['CustomerId','FirstName','LastName','Total']]
customer_total = all.groupby(['CustomerId', 'FirstName','LastName'])['Total'].sum().reset_index().sort_values(by='Total', ascending=False).head(5)
customer_total

,CustomerId,FirstName,LastName,Total
5,6,Helena,Holý,49.62
25,26,Richard,Cunningham,47.62
56,57,Luis,Rojas,46.62
44,45,Ladislav,Kovács,45.62
45,46,Hugh,O'Reilly,45.62


In [ ]:
import sqlite3
import pandas as pd

# Baza bilan ulanish
conn = sqlite3.connect("chinook.db")

# 1. InvoiceLine va Track jadvallarini yuklash
invoice_lines = pd.read_sql_query("SELECT * FROM InvoiceLine", conn)
tracks = pd.read_sql_query("SELECT TrackId, AlbumId FROM Track", conn)

# 2. InvoiceLine + Track (AlbumId olish uchun)
invoice_lines = invoice_lines.merge(tracks, on='TrackId', how='left')

# 3. Har bir albomda nechta trek bor?
total_tracks_per_album = tracks.groupby('AlbumId').agg(TotalTracks=('TrackId', 'count')).reset_index()

# 4. Mijozning har bir albomdan nechta trek olganini topish
customer_album_purchases = invoice_lines.groupby(['CustomerId', 'AlbumId']).agg(PurchasedTracks=('TrackId', 'count')).reset_index()

# 5. Har bir albom uchun umumiy trek sonini birlashtirish
customer_album_purchases = customer_album_purchases.merge(total_tracks_per_album, on='AlbumId', how='left')

# 6. To‘liq albom olganini aniqlash
customer_album_purchases['IsFullAlbum'] = customer_album_purchases['PurchasedTracks'] >= customer_album_purchases['TotalTracks']

# 7. Har bir mijoz uchun - hech bo‘lmaganda bitta full albom bormi?
full_album_buyers = customer_album_purchases.groupby('CustomerId')['IsFullAlbum'].max().reset_index()
full_album_buyers['Preference'] = full_album_buyers['IsFullAlbum'].apply(lambda x: 'Full Album Buyer' if x else 'Individual Track Buyer')

# 8. Yakuniy statistika
summary = full_album_buyers['Preference'].value_counts().reset_index()
summary.columns = ['Preference', 'Customer Count']
summary['Percentage'] = round((summary['Customer Count'] / summary['Customer Count'].sum()) * 100, 2)

conn.close()
print(summary)
